In [148]:
import pandas as pd
import re
import string

In [149]:
df = pd.read_csv(r'../data/GTZAN/names_dataframe_complete.csv')
df

,filename,artist,title,genre,lyrics
0,blues.00000.wav,John Lee Hooker,"One Bourbon, One Scotch And One Beer",blues,NaN
1,blues.00001.wav,John Lee Hooker,I'm In The Mood For Love,blues,NaN
2,blues.00002.wav,John Lee Hooker,Think Twice Before You Go,blues,NaN
3,blues.00003.wav,John Lee Hooker,I'm Bad Like Jesse James,blues,NaN
4,blues.00004.wav,John Lee Hooker,Walkin’ The Boogie,blues,NaN
...,...,...,...,...,...
995,rock.00095.wav,The Stone Roses,Bye Bye Bad Man,rock,NaN
996,rock.00096.wav,The Stone Roses,(Song For My) Sugar Spun Sister,rock,NaN
997,rock.00097.wav,The Stone Roses,Made Of Stone,rock,NaN
998,rock.00098.wav,The Stone Roses,Shoot You Down,rock,NaN


In [150]:
print(set(df.genre))

{'metal', 'hiphop', 'pop', 'reggae', 'disco', 'classical', 'country', 'blues', 'jazz', 'rock'}


In [151]:
df['genre'].value_counts()

blues        100
classical    100
country      100
disco        100
hiphop       100
jazz         100
metal        100
pop          100
reggae       100
rock         100
Name: genre, dtype: int64

In [152]:
print(df[df['title'].isna()])
indices = df[df['title'].isna()].index.sort_values(ascending=False)

for i in indices:
  df.drop(index=i, inplace=True)

df[df['title'].isna()]

                filename        artist title      genre  lyrics
105  classical.00005.wav       JS Bach   NaN  classical     NaN
106  classical.00006.wav       JS Bach   NaN  classical     NaN
107  classical.00007.wav       JS Bach   NaN  classical     NaN
108  classical.00008.wav       JS Bach   NaN  classical     NaN
109  classical.00009.wav       JS Bach   NaN  classical     NaN
..                   ...           ...   ...        ...     ...
853     reggae.00053.wav           NaN   NaN     reggae     NaN
862     reggae.00062.wav           NaN   NaN     reggae     NaN
865     reggae.00065.wav  Dennis Brown   NaN     reggae     NaN
879     reggae.00079.wav           NaN   NaN     reggae     NaN
890     reggae.00090.wav           NaN   NaN     reggae     NaN

[73 rows x 5 columns]


,filename,artist,title,genre,lyrics


In [153]:
"""df.iloc[202].artist = 'Hank Williams'
df.iloc[231].artist = 'George Strait'
df.iloc[238].artist = 'Donna Fargo & Johnny Rodriguez'
df.iloc[357].artist = 'Boney M.'
df.iloc[710].artist = 'Robert Downey Jr.'
df.iloc[710].title = 'Every Breath You Take'
df.iloc[714].artist = 'The Police'
df.iloc[714].title = 'Every Breath You Take'
df.iloc[783].artist = 'Madonna'"""

"df.iloc[202].artist = 'Hank Williams'\ndf.iloc[231].artist = 'George Strait'\ndf.iloc[238].artist = 'Donna Fargo & Johnny Rodriguez'\ndf.iloc[357].artist = 'Boney M.'\ndf.iloc[710].artist = 'Robert Downey Jr.'\ndf.iloc[710].title = 'Every Breath You Take'\ndf.iloc[714].artist = 'The Police'\ndf.iloc[714].title = 'Every Breath You Take'\ndf.iloc[783].artist = 'Madonna'"

## Testing out different APIs for getting lyrics

In [154]:
from musixmatch import Musixmatch
import musixmatch_api_key

musixmatch = Musixmatch(musixmatch_api_key.client_access_token)

In [155]:
song = musixmatch.matcher_lyrics_get(q_track = "Hips Don't Lie", q_artist = "Shakira")
# free API key only gives 30% of lyrics
print(song['message']['body']['lyrics']['lyrics_body'])

Ladies up in here tonight
No fighting (We got the refugees up in here)
No fighting
No fighting
Shakira, Shakira

I never really knew that she could dance like this
She makes a man wanna speak Spanish
¿Cómo se llama? (Sí), bonita (Sí)
Mi casa, su casa (Shakira, Shakira)

Oh, baby, when you talk like that
You make a woman go mad
So be wise (Sí) and keep on (Sí)
Reading the signs of my body (Uno, dos, tres, cuatro)

I'm on tonight, you know my hips don't lie
And I'm starting to feel it's right
All the attraction, the tension
Don't you see, baby? This is perfection

Hey, girl, I can see your body moving
And it's driving me crazy
And I didn't have the slightest idea
Until I saw you dancing
The way you walk up on the dance floor
Nobody cannot ignore the way you move your body, girl
And everything's so unexpected
The way you right and left it
So you can keep on shaking it (Let's go)

I never really knew that she could dance like this
She makes a man want to speak Spanish
¿Cómo se llama? (Sí),

In [156]:
import azapi

API = azapi.AZlyrics()

API.artist = "Shakira"
API.title = "Hips Don't Lie"
API.getLyrics(save=False)

print(API.lyrics)

Ladies up in here tonight
No fighting
We got the refugees up in here
No fighting, no fighting

Shakira, Shakira

I never really knew that she could dance like this
She make a man wanna speak Spanish
¿Cómo se llama (sí), bonita (sí), mi casa (sí, Shakira Shakira), su casa

Oh baby when you talk like that
You make a woman go mad
So be wise (Sí!) and keep on (Sí!)
Reading the signs of my body
(Uno, dos, tres, cuatro)

I'm on tonight
You know my hips don't lie
And I'm starting to feel it's right
All the attraction, the tension
Don't you see baby? this is perfection

Hey girl, I can see your body moving
And it's driving me crazy
And I didn't have the slightest idea
Until I saw you dancing

And when you walk up on the dance floor
Nobody cannot ignore the way you move your body, girl (Just smooth)
And everything's so unexpected – the way you right and left it
So you can keep on shaking it (Let's go)

I never really knew that she could dance like this
She make a man wanna speak Spanish
¿Cómo s

In [157]:
from lyricsgenius import Genius
import genius_api_key

genius = Genius(genius_api_key.client_access_token)

In [158]:
def clean_lyrics(lyrics):
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    
    # remove header
    index = lyrics.find('Lyrics')
    if index > -1:
        lyrics = lyrics[index+len('Lyrics'):]

    # remove end
    index = lyrics.find('Embed')
    if index > -1:
        lyrics = lyrics[:index]
    lyrics = lyrics.rstrip(string.digits)
    
    return lyrics

In [159]:
song = genius.search_song(artist="Shakira", title="Hips Don't Lie")
lyrics = clean_lyrics(song.lyrics)
print(lyrics)

Searching for "Hips Don't Lie" by Shakira...
Done.

Ladies up in here tonight
No fighting 
No fighting
No fighting
Shakira, Shakira


I never really knew that she could dance like this 
She make a man wanna speak Spanish
¿Cómo se llama , bonita? 
Mi casa, su casa 
Oh baby, when you talk like that  
You make a woman go mad 
So be wise  and keep on 
Reading the signs of my body 
No fighting

I'm on tonight
You know my hips don't lie 
And I'm starting to feel it's right
All the attraction, the tension
Don't you see, baby? This is perfection


Hey girl, I can see your body moving
And it's driving me crazy 
And I didn't have the slightest idea
Until I saw you dancing 
That when you walk up on the dance floor, nobody cannot ignore
The way you move your body, girl 
And everything's so unexpected, the way you right and left it
So you can keep on shaking it 


I never really knew that she could dance like this 
She make a man wanna speak Spanish
¿Cómo se llama , bonita? 
Mi casa, su casa 
Oh ba

## Scrape lyrics for all songs

In [160]:
for idx in df.index:
    song = genius.search_song(artist=df.artist[idx], title=df.title[idx])
    if song:
        lyrics = clean_lyrics(song.lyrics)
        #print(lyrics)
        df.lyrics[idx] = lyrics
    else:
        print("Song not found")

Searching for "One Bourbon, One Scotch And One Beer" by John Lee Hooker...
Done.
Searching for "I'm In The Mood For Love" by John Lee Hooker...


/tmp/ipykernel_18151/699782468.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.lyrics[idx] = lyrics


Done.
Searching for "Think Twice Before You Go" by John Lee Hooker...
Done.
Searching for "I'm Bad Like Jesse James" by John Lee Hooker...
Done.
Searching for "Walkin’ The Boogie" by John Lee Hooker...
Done.
Searching for "Sugar Mama" by John Lee Hooker...
Done.
Searching for "Dreamin' Blues" by John Lee Hooker...
Done.
Searching for "Ground Hog Blues" by John Lee Hooker...
Done.
Searching for "Shake It Baby" by John Lee Hooker...
Done.
Searching for "It's My Own Fault" by John Lee Hooker...
Done.
Searching for "Lonely Boy Boogie" by John Lee Hooker...
Done.
Searching for "It Serves Me Right To Suffer" by John Lee Hooker...
Done.
Searching for "Cross Road Blues" by Robert Johnson...
Done.
Searching for "Terraplane Blues" by Robert Johnson...
Done.
Searching for "Come On In My Kitchen" by Robert Johnson...
Done.
Searching for "Walking blues" by Robert Johnson...
Done.
Searching for "Last Fair Deal Gone Down" by Robert Johnson...
Done.
Searching for "32-20 Blues" by Robert Johnson...
Don

KeyboardInterrupt: 

In [161]:
df

,filename,artist,title,genre,lyrics
0,blues.00000.wav,John Lee Hooker,"One Bourbon, One Scotch And One Beer",blues,"One bourbon, one scotch and one beer\nOne bour..."
1,blues.00001.wav,John Lee Hooker,I'm In The Mood For Love,blues,A Valentine’s Playlist is an important part of...
2,blues.00002.wav,John Lee Hooker,Think Twice Before You Go,blues,"\nThink twice before you go, baby, this will b..."
3,blues.00003.wav,John Lee Hooker,I'm Bad Like Jesse James,blues,"\n...A little thing I'm going to do called ""I'..."
4,blues.00004.wav,John Lee Hooker,Walkin’ The Boogie,blues,"Well my mama she didn't 'low me, just to stay ..."
...,...,...,...,...,...
995,rock.00095.wav,The Stone Roses,Bye Bye Bad Man,rock,NaN
996,rock.00096.wav,The Stone Roses,(Song For My) Sugar Spun Sister,rock,NaN
997,rock.00097.wav,The Stone Roses,Made Of Stone,rock,NaN
998,rock.00098.wav,The Stone Roses,Shoot You Down,rock,NaN
